In [1]:
from get_financial import *

In [2]:
gicode = '000270'
get_html(gicode)

In [16]:
import pandas as pd
# AI에게 넘길 데이터 정리
fs = get_stock_items(gicode)

# 연간재무제표
stock_name = fs['name']
annual_financial_statements = pd.DataFrame(fs['year'])
annual_financial_statements.set_index('year_chk', inplace=True)
annual_financial_statements['영업이익률'] = (annual_financial_statements['영업이익']/annual_financial_statements['매출액'])*100
annual_financial_statements['당기순이익률'] = (annual_financial_statements['당기순이익']/annual_financial_statements['매출액'])*100
annual_financial_statements.rename(columns={'PER수정주가': 'PER', 'PBR수정주가': 'PBR'}, inplace=True)

# 분기재무제표
quarterly_financial_statements = pd.DataFrame(fs['quarter'])
quarterly_financial_statements.set_index('year_chk', inplace=True)
quarterly_financial_statements['영업이익률'] = (quarterly_financial_statements['영업이익']/quarterly_financial_statements['매출액'])*100
quarterly_financial_statements['당기순이익률'] = (quarterly_financial_statements['당기순이익']/quarterly_financial_statements['매출액'])*100
quarterly_financial_statements.rename(columns={'PER수정주가': 'PER', 'PBR수정주가': 'PBR'}, inplace=True)

# 기준 컬럼
columns = ['매출액', '영업이익','영업이익률', '당기순이익', 'ROE', 'PER', 'PBR']

In [17]:
# 연간재무제표
annual_financial_statements[columns]

,매출액,영업이익,영업이익률,당기순이익,ROE,PER,PBR
year_chk,,,,,,,
2020/12,591681,20665,3.492591,14876,5.05,17.00,0.84
2021/12,698624,50657,7.250968,47603,14.69,7.00,0.95
2022/12,865590,72331,8.356266,54090,14.57,4.44,0.61
2023/12(E),981604,108005,11.002910,83100,19.37,4.05,0.72


In [19]:
# 분기별재무제표
quarterly_financial_statements[columns]

,매출액,영업이익,영업이익률,당기순이익,ROE,PER,PBR
year_chk,,,,,,,
2023/03,236907,28740,12.131343,21198,21.36,None,0.81
2023/06(E),255224,29801,11.676410,21578,23.40,None,NaN
2023/09(E),241379,24001,9.943284,17979,NaN,None,NaN
2023/12(E),248273,25378,10.221812,17781,NaN,None,NaN


In [29]:
# 재무상태표
ftable = fs['daecha_table']
col_name = [date for date, value in ftable['자산']]
row_date = {key: [v for d, v in values] for key, values in ftable.items()}
row_date['year_chk'] = col_name
financial_statements = pd.DataFrame(row_date)
financial_statements.set_index('year_chk', inplace=True)
financial_statements['부채비율'] = (financial_statements['부채']/financial_statements['자본'])*100
financial_statements

,자산,부채,자본,부채비율
year_chk,,,,
2020/12,604904,305988,298917,102.365540
2021/12,668500,319374,349126,91.478148
2022/12,737110,343679,393431,87.354326
2023/03,776127,375811,400316,93.878586


In [31]:
# 현금흐름표
cash_table = fs['cash_table']
col_name = [date for date, value in cash_table['영업활동으로인한현금흐름']]
row_date = {key: [v for d, v in values] for key, values in cash_table.items()}
row_date['year_chk'] = col_name
statement_of_cash_flows = pd.DataFrame(row_date)
statement_of_cash_flows.set_index('year_chk', inplace=True)
statement_of_cash_flows

,영업활동으로인한현금흐름,투자활동으로인한현금흐름,재무활동으로인한현금흐름
year_chk,,,
2020/12,54239,-28649,35173
2021/12,73597,-44239,-16205
2022/12,93332,-56713,-34544
2023/03,24378,8426,-8274


In [20]:
fs

{'name': '기아',
 'stxt': ['KSE  코스피 운수장비', 'FICS  자동차', 'K200'],
 'current': {'current_price': 81400, 'base_date': '2023/06/12 14:57'},
 'summary': ['1999년 아시아자동차와 함께 현대자동차에 인수되었고, 기아차판매, 아시아자동차, 기아대전판매, 아시아차판매 등 4개사를 통합함. 국내(소하리, 화성, 광주, 위탁)와 미국, 슬로박, 멕시코, 인도 공장에서 생산하고 있으며, 모닝의 경우 동희오토 서산공장에서 위탁 생산하고 있음. 전체 매출액의 약 40% 내외를 내수시장에서 판매하고 있으며, 북중미 및 유럽시장 등 해외에서의 판매비중은 약 60% 수준임.',
  '2022년 12월 전년동기 대비 연결기준 매출액은 23.9% 증가, 영업이익은 42.8% 증가, 당기순이익은 13.6% 증가. 북중미 및 유럽 등 해외시장에서의 매출 증가와 더불어 내수 선전으로 외형 비교적 큰 폭으로 증가. 원가율 또한 하락하며 수익성 또한 크게 확대된 모습. 올해는 우수한 품질 평가를 바탕으로 전기차 시장에서 성장세를 이어갈것으로 기대하고, 해외시장에서의 판매량 또한 확대될 것으로 기대함.'],
 'common': 402044203,
 'market_capacity': 33329500000000,
 'treasury': 6962511,
 'year': {'매출액': {'c1': 591681, 'c2': 698624, 'c3': 865590, 'c4': 981604},
  '영업이익': {'c1': 20665, 'c2': 50657, 'c3': 72331, 'c4': 108005},
  '영업이익_1': {'c1': 20665, 'c2': 50657, 'c3': 72331, 'c4': None},
  '당기순이익': {'c1': 14876, 'c2': 47603, 'c3': 54090, 'c4': 83100},
  '지배주주순이익': {'c1': 14876, 'c2': 4